In [ ]:
!pip install spacy
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00


In [ ]:
import spacy
import unicodedata
from datasets import load_dataset

# Load the dataset from Hugging Face Hub
ds = load_dataset("text", data_files={"train": "babylm_sents.txt"})

# Load the spaCy model
nlp = spacy.load("en_core_web_trf")



Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:


# Optionally enable GPU if available
if spacy.prefer_gpu():
    print("GPU enabled.")
else:
    print("Running on CPU.")

# Load your spaCy model (ensure it supports GPU if you enabled it)
nlp = spacy.load("en_core_web_sm")

def pos_tag_sentences(examples):
    # Increase batch_size for spaCy's pipe if memory allows
    docs = list(nlp.pipe(examples["text"], batch_size=1024))
    pos_tags = [
        " ".join(f"{token.text}|{token.pos_}" for token in doc).strip()
        for doc in docs
    ]
    return {"pos_tags": pos_tags}

# Assuming ds is your dataset, use a larger batch size and multiple processes
ds_encoded = ds.map(pos_tag_sentences, batched=True, batch_size=1024, num_proc=1)

GPU enabled.


Map:   0%|          | 0/11632617 [00:00<?, ? examples/s]

In [ ]:
ds_encoded.save_to_disk("/content/dataset_with_pos_tags_and_more")

print("POS tagging completed. Encoded dataset saved to: /content/dataset_with_pos_tags_and_more")

# Optional: Print a sample of the POS-tagged sentences from the training split
for sentence, pos in zip(ds["train"]["text"][:3], ds_encoded["train"]["pos_tags"][:3]):
    print("Sentence:", sentence)
    print("POS Tags:", pos)


Saving the dataset (0/4 shards):   0%|          | 0/11632617 [00:00<?, ? examples/s]

POS tagging completed. Encoded dataset saved to: /content/dataset_with_pos_tags_and_more
Sentence: It may not look like much now but this is actually going to turn into one of the three student lounges which is a really big step up from our old campus.
POS Tags: It|PRON may|AUX not|PART look|VERB like|ADP much|ADJ now|ADV but|CCONJ this|PRON is|AUX actually|ADV going|VERB to|PART turn|VERB into|ADP one|NUM of|ADP the|DET three|NUM student|NOUN lounges|NOUN which|PRON is|AUX a|DET really|ADV big|ADJ step|NOUN up|ADP from|ADP our|PRON old|ADJ campus|NOUN .|PUNCT
Sentence: Right beside me is going to be the future home of the Nittany Lion.
POS Tags: Right|ADV beside|ADP me|PRON is|AUX going|VERB to|PART be|AUX the|DET future|ADJ home|NOUN of|ADP the|DET Nittany|PROPN Lion|PROPN .|PUNCT
Sentence: Well that's it for our first look at our new building, I hope you're as excited as I am, and I hope to come back in a month or so and give you an updated look at our new building.
POS Tags: Well|I

In [ ]:
import shutil

# Zip the folder
shutil.make_archive("/content/dataset_with_pos_tags_and_more", 'zip', "/content/dataset_with_pos_tags_and_more")

# Download the zip file
from google.colab import files
files.download("/content/dataset_with_pos_tags_and_more.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>